In [ ]:
%load_ext autoreload
%autoreload 2

# Find Test Set

In [ ]:
from src.data import load_unosat_labels, load_unosat_aois
from src.data.utils import aoi_to_city

labels_to_keep = [1,2]
aois = load_unosat_aois()
aois['city'] = aois['aoi'].apply(aoi_to_city)
labels = load_unosat_labels(labels_to_keep=labels_to_keep)
grouped = labels.groupby(['aoi', 'damage']).size().reset_index(name='counts')
df_count = grouped.pivot(index='aoi', columns='damage', values='counts').fillna(0).astype(int)
df_count['total'] = df_count.sum(axis=1)
for l in labels_to_keep:
    df_count[f'{l} (%)'] = df_count[l] / df_count[l].sum() * 100
df_count['total (%)'] = df_count['total'] / df_count['total'].sum() * 100
print('Number of labels per AOI and damage level:')
df_count.round(2)

## Old one

F1 = 0.859 (labels=[1,2])

F1 = 0.810 (labels=[1,2,3])

In [ ]:
aois_test_old = ['UKR6', 'UKR8', 'UKR12', 'UKR15']
n_test = df_count.loc[aois_test_old].sum()
n_train = df_count.drop(aois_test_old).sum()
n_tot = df_count.sum()

for d in labels_to_keep:
    print(f"Damage {d}:")
    print(f"  - Train: {n_train[d]} ({n_train[f'{d} (%)']:.2f}%)")
    print(f"  - Test: {n_test[d]} ({n_test[f'{d} (%)']:.2f}%)")
print('Total')
print(f"  - Train: {n_train['total']} ({n_train['total (%)']:.2f}%)")
print(f"  - Test: {n_test['total']} ({n_test['total (%)']:.2f}%)")

In [ ]:
def my_colormap(value):  # scalar value defined in 'column'
    if value >= 1:
        return "orange"
    return "blue"

aois_ = aois.copy()
aois_.geometry = aois_.geometry.centroid
aois_['is_test'] = aois_['aoi'].isin(aois_test_old).astype(int)
aois_['color'] = aois_['is_test'].apply(my_colormap)

# Explore dataframe based on column 'is_test'. Color is_test==1 in green and is_test==0 in red
aois_.explore(color=aois_['color'], style_kwds={'weight':10})

## New one

In [ ]:
aois_test = [f"UKR{i}" for i in range(1,19) if i not in [1,2,3,4]]#["UKR6", "UKR7", "UKR8", "UKR10", "UKR12", "UKR14"]
n_test = df_count.loc[aois_test].sum()
n_train = df_count.drop(aois_test).sum()
n_tot = df_count.sum()

for d in labels_to_keep:
    print(f"Damage {d}:")
    print(f"  - Train: {n_train[d]} ({n_train[f'{d} (%)']:.2f}%)")
    print(f"  - Test: {n_test[d]} ({n_test[f'{d} (%)']:.2f}%)")
print('Total')
print(f"  - Train: {n_train['total']} ({n_train['total (%)']:.2f}%)")
print(f"  - Test: {n_test['total']} ({n_test['total (%)']:.2f}%)")

In [ ]:
aois_ = aois.copy()
aois_.geometry = aois_.geometry.centroid
aois_['is_test'] = aois_['aoi'].isin(aois_test).astype(int)
aois_['color'] = aois_['is_test'].apply(my_colormap)

# Explore dataframe based on column 'is_test'. Color is_test==1 in green and is_test==0 in red
aois_.explore(color=aois_['color'], style_kwds={'weight':10})

In [ ]:
from src.postprocessing.preds_buildings import load_buildings_with_preds
run_name = "240212"
aoi = "UKR2"
gdf = load_buildings_with_preds(aoi, run_name)